In [2]:
import pandas as pd
import os
desktop_path = os.path.expanduser("~/Desktop")

playlist_file = os.path.join(desktop_path, "spotify_dataset.csv")
song_attr_file = os.path.join(desktop_path, "Spotify_Song_Attributes.csv")

playlist_df = pd.read_csv(playlist_file, on_bad_lines='skip')
song_attr_df = pd.read_csv(song_attr_file)

playlist_df.describe()

FileNotFoundError: [Errno 2] No such file or directory: '/root/Desktop/spotify_dataset.csv'

In [ ]:
#fixing column names (removing spaces, quotes, uppercases)
playlist_df.columns = (
    playlist_df.columns
    .str.replace('"', '')
    .str.strip()
    .str.lower()
)
print(playlist_df.columns.tolist())


In [ ]:
#cleaning the dataset (removing nulls and duplicates)
new_playlist_df = playlist_df.dropna(subset=['artistname', 'trackname']).copy()

new_playlist_df['artistname'] = new_playlist_df['artistname'].str.strip()
new_playlist_df['trackname'] = new_playlist_df['trackname'].str.strip()

new_playlist_df = new_playlist_df.drop_duplicates(subset=['artistname', 'trackname'])
new_playlist_df = new_playlist_df.reset_index(drop=True)

print(new_playlist_df.describe())

In [ ]:
new_playlist_df.shape

In [ ]:
desktop_path = os.path.expanduser("~/Desktop")

filename = "cleaned_spotify_playlists.csv"

new_playlist_df.to_csv(os.path.join(desktop_path, filename), index=False)

In [3]:
#cleaning dataset 2
song_attr_df.dropna(subset=['artistName', 'genre', 'msPlayed', 'danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','type','id','uri','track_href','analysis_url','duration_ms','time_signature'], inplace=True)  # Drop rows with missing critical values

song_attr_df.drop_duplicates(inplace=True)

for col in song_attr_df.select_dtypes(include=['object']).columns:
    song_attr_df[col] = song_attr_df[col].astype(str).str.lower().str.strip()

song_attr_df.columns = [col.strip().lower().replace(" ", "_") for col in song_attr_df.columns]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numeric_cols = song_attr_df.select_dtypes(include=['number']).columns
song_attr_df[numeric_cols] = scaler.fit_transform(song_attr_df[numeric_cols])
song_attr_df = song_attr_df.apply(lambda x: x.str.encode('utf-8').str.decode('utf-8') if x.dtype == "object" else x)

print(song_attr_df)

song_attr_df.to_csv("cleaned_dataset.csv", index=False)

print("Dataset cleaned and saved as 'cleaned_dataset.csv'")

print(song_attr_df.head())

NameError: name 'song_attr_df' is not defined

In [ ]:
def merge_datasets(new_playlist_df, song_attr_df):
    """
    Clean and merge the playlist and attributes datasets on track name and artist name.
    Returns the merged DataFrame.
    """

    # Merge on track name + artist name
    merged_df = pd.merge(
        new_playlist_df,
        song_attr_df,
        on=['track_name'],
        how='inner'  # or 'left' depending on what you want to keep
    )

    return merged_df
merge_datasets(new_playlist_df, song_attr_df)

In [ ]:

from gensim.models import Word2Vec

# Create unique song identifier
df['song_id'] = df['track_name'].str.lower() + ' - ' + df['artist_name'].str.lower()

# Group songs by playlist
playlists = df.groupby('playlistname')['song_id'].apply(list).tolist()

# Train Word2Vec model
model = Word2Vec(
    sentences=playlists,
    vector_size=100,
    window=5,
    min_count=1,
    sg=1,
    workers=4,
    epochs=20
)

# Save model
model.save("song_word2vec.model")

# Example: Find similar songs
song_query = 'blinding lights - the weeknd'
if song_query in model.wv:
    similar_songs = model.wv.most_similar(song_query, topn=10)
    for song, score in similar_songs:
        print(f"{song} (score: {score:.3f})")
else:
    print("Song not in vocabulary.")


In [ ]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Example user input
user_preferences = {
    'genre': 'pop',
    'mood': 'happy',  # high valence
    'energy': 0.7,
    'danceability': 0.6,
    'favorite_song': 'blinding lights'  # example seed
}

# Map mood to valence threshold
mood_valence_map = {
    'happy': 0.6,
    'sad': 0.3,
    'energetic': 0.7,
    'calm': 0.4
}
valence_threshold = mood_valence_map.get(user_preferences['mood'], 0.5)

# Make a full song key
merged_df['song_key'] = merged_df['track_name'].str.lower().str.strip()

# Check if the favorite song is in the model
seed_song = user_preferences['favorite_song'].lower().strip()
if seed_song not in w2v_model.wv:
    print(f"'{seed_song}' not in vocabulary. Please choose another song.")
else:
    similar_songs = w2v_model.wv.most_similar(seed_song, topn=500)
    similar_song_names = [song for song, sim in similar_songs]
    filtered_df = merged_df[merged_df['song_key'].isin(similar_song_names)].copy()

    # Apply additional metadata filters
    filtered_df = filtered_df[
        (filtered_df['genre'].str.lower() == user_preferences['genre'].lower()) &
        (filtered_df['valence'] >= valence_threshold) &
        (filtered_df['energy'] >= user_preferences['energy']) &
        (filtered_df['danceability'] >= user_preferences['danceability'])
    ]

    # Rank songs by Word2Vec similarity score
    similarity_scores = []
    for song in filtered_df['song_key']:
        sim_score = w2v_model.wv.similarity(seed_song, song)
        similarity_scores.append(sim_score)

    filtered_df['similarity'] = similarity_scores

    # Sort by similarity
    ranked_df = filtered_df.sort_values(by='similarity', ascending=False)

    # Show top 10 recommendations
    recommendations = ranked_df[['track_name', 'artist_name', 'genre', 'valence', 'similarity']].head(10)
    print("Top Recommendations:")
    print(recommendations)
